# Preparação de Dados para Machine Learning

Este notebook prepara o dataset para treinar um modelo de machine learning. As etapas seguem o `guia_preparacao_machine_learning.md`.

Etapas:
1. Limpeza e pré-processamento (valores ausentes, coluna 'Age').
2. Remoção de duplicatas.
3. Tratamento de outliers (IQR).
4. Transformação de variáveis assimétricas (log).
5. Remoção de variáveis altamente correlacionadas.
6. Agrupamento de categorias raras.
7. Encoding de variáveis categóricas (One-Hot Encoding).
8. Normalização de variáveis numéricas (StandardScaler).

O resultado será salvo em `data/train_data_ml_v2.csv`.

## 1. Importação das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

## 2. Carregamento dos Dados

In [ ]:
df = pd.read_csv('../data/train_data.csv')

## 3. Limpeza e Pré-processamento

Antes de seguir o guia, realizamos uma limpeza inicial.

### 3.1. Tratamento de Valores Ausentes

In [ ]:
df['Bed Grade'] = df['Bed Grade'].fillna(df['Bed Grade'].mode()[0])
df['City_Code_Patient'] = df['City_Code_Patient'].fillna(0)

### 3.2. Limpeza da Coluna 'Age'

A coluna 'Age' está em formato de string ('51-60'). Convertemos para um valor numérico usando a média do intervalo.

In [ ]:
def age_to_numeric(age_range):
    if isinstance(age_range, str):
        low, high = age_range.split('-')
        return (int(low) + int(high)) / 2
    return age_range

df['Age'] = df['Age'].apply(age_to_numeric)

## 4. Passos do Guia de Preparação

### 4.1. Remoção de Duplicatas

In [ ]:
df = df.drop_duplicates()

### 4.2. Tratamento de Outliers (IQR)

In [ ]:
num_cols_original = df.select_dtypes(include=np.number).columns.tolist()
for col in num_cols_original:
    if col not in ['case_id', 'patientid']:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lim_inf = Q1 - 1.5 * IQR
        lim_sup = Q3 + 1.5 * IQR
        df = df[(df[col] >= lim_inf) & (df[col] <= lim_sup)]

### 4.3. Transformação de Variáveis Assimétricas

In [ ]:
skewed_cols = ['Admission_Deposit'] # Exemplo
for col in skewed_cols:
    if col in df.columns:
      df[col] = np.log1p(df[col])

### 4.4. Remoção de Variáveis Altamente Correlacionadas

In [ ]:
num_cols_clean = df.select_dtypes(include=np.number).columns.tolist()
corr_matrix = df[num_cols_clean].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
df = df.drop(columns=to_drop)

### 4.5. Agrupamento de Categorias Raras

In [ ]:
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    if col != 'Stay': # Não agrupar a variável alvo
        freq = df[col].value_counts(normalize=True)
        raras = freq[freq < 0.01].index
        if len(raras) > 0:
            df[col] = df[col].replace(raras, 'OUTRA')

### 4.6. Encoding de Variáveis Categóricas (One-Hot)

In [ ]:
target_col = 'Stay'
categorical_features = df.select_dtypes(include=['object']).columns.drop(target_col, errors='ignore')
df = pd.get_dummies(df, columns=categorical_features, drop_first=True)

### 4.7. Normalização das Variáveis Numéricas

In [ ]:
numerical_features = df.select_dtypes(include=np.number).columns.drop(['case_id', 'patientid'], errors='ignore')
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

## 5. Salvando o Dataset Processado

In [ ]:
df.to_csv('../data/train_data_ml_v2.csv', index=False)